# 1. query()

- query_texts: 查询的文本
- query_embeddings: 根据embeddings来查询
- n_results: 返回最匹配的n个文档数据
- where: 根据metadata过滤
- where_document: 根据关键字从文档字符串中过滤。
- include: 默认: metadatas,documents,distances, 可选值：documents/embeddings/metadatas/distances/uris/data


```python
results = collection.query(
    query_texts=["doc10", "thus spake zarathustra", ...],
    query_embeddings: [],
    n_results=10,
    where={"metadata_field": "is_equal_to_this"},
    where_document={"$contains": "search_string"}
)
```


In [8]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import chromadb
import uuid

# 数据准备
origin_docs = [
    {
        "title":"科技行业2025年展望",
        "path": "./科技行业 2025 年展望.txt",
        "author": "沈岱;马智焱;黄佳琦",
        "time": "2024年12月13日"
    },
    {
        "title":"DeepSeek突传大消息！最新回应来了",
        "path": "./deepseek.txt",
        "author": "券商中国",
        "time": "2025年2月26日"
    },
]

# 文档加载和切割
def get_splits(file_path: str):
    text_loader = TextLoader(file_path)
    docs = text_loader.load()

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=100)
    all_splits = text_splitter.split_documents(docs)
    return all_splits


async def store_data():
    client = await chromadb.AsyncHttpClient(host="localhost", port=8000)
    collection = await client.get_or_create_collection(name="query_video_collection")
    for doc in origin_docs:
        all_splits = get_splits(doc["path"])
        print(len(all_splits))
        print("*" * 80)
        ids = [uuid.uuid4().hex for _ in all_splits]
        documents = [doc.page_content for doc in all_splits]
        # [(0, item1), (1, item2), ...]
        metadatas = [{**doc, "index": index} for index, _ in enumerate(all_splits)]
        await collection.add(ids=ids, documents=documents, metadatas=metadatas)

await store_data()

23
********************************************************************************
11
********************************************************************************


In [9]:
client = await chromadb.AsyncHttpClient(host="localhost", port=8000)
collection = await client.get_or_create_collection(name="query_video_collection")
await collection.count()

34

## 1.1 query_embeddings

根据向量来查询


In [15]:
# from chromadb.utils import embedding_functions

# default_ef = embedding_functions.DefaultEmbeddingFunction()

# query_embeddings = default_ef(["中东非的智能手机出货量为多少部?"])
# # query_embeddings
# results = await collection.query(
#     query_embeddings=query_embeddings,
#     n_results=20,
# )
# results

In [16]:
results = await collection.query(
    query_texts=["中东非的智能手机出货量为多少部?"],
    n_results=20,
)
results

{'ids': [['0fdfe513c3794565b865a104945105d5',
   '5d8b773d740c459aaa2ed01e8365d4b7',
   '8bafa6ead2aa4ec1b00a31b912d0ba86',
   'dc47b8a98d444ff8983f2eaaf6ce01d6',
   '0c86b133f7514665b4a8684211d2d3d1',
   'f7a67861b5e24da3afe0c593e206afcd',
   'b71b2441926a4697a2d3faf22688b479',
   'ba044e3ef2ec4510b48d691cf81c75a2',
   '00b9ee79053942bcaf4aaaeaa1a26fc4',
   '34ad146d37a449c18b62dd8f528c71e4',
   'c549d0bc2d2f4505824171d42ac3681c',
   '12b303af297347e299251f3d8a79a549',
   '33bbdacd6c3f4a43bff29307eb8c9ad0',
   'bee707c146cf4146ad68786fabeb91ee',
   '982b696ee125406aabee32d3b86ffe62',
   'e887f25d05354c40b27151d7feff5586',
   'ccf537ac31cf4818a3b2434899a65e94',
   '68d37f062d0a4213840d20eec87bfbe4',
   'ccb49d54bac24338a92373512e544be7',
   '5167bd429e6748a19b3a911990475500']],
 'distances': [[0.6475816981625376,
   0.6794282086911847,
   0.7959771981980448,
   0.8101529216182416,
   0.9953305774166606,
   1.00379554307,
   1.028614060939762,
   1.0816273847301765,
   1.093567293739339

## 1.2 where

根据metadata过滤


### 1.2.1 简单语法

Operator：\\$eq(等于) / \\$ne(不等于) / \\$gt(大于) / \\$gte(大于等于) / \\$lt(小于) / \\$lte(小于等于)

```json
{
    "metadata_field": {
        <Operator>: <Value>
    }
}
```
等于（\\$eq）的时候，有个快捷语法：

```python
{
    "metadata_field": "search_string"
}

# is equivalent to

{
    "metadata_field": {
        "$eq": "search_string"
    }
}

```

In [19]:
# results = await collection.query(
#     query_texts=["中东非的智能手机出货量为多少部?"],
#     n_results=20,
#     where={
#         "title": "科技行业2025年展望"
#     }
# )
# results

### 1.2.2 \\$and和 \\$or查询

```python
# 1. $and
{
    "$and": [
        {
            "metadata_field": {
                <Operator>: <Value>
            }
        },
        {
            "metadata_field": {
                <Operator>: <Value>
            }
        }
    ]
}
# 2. $or
{
    "$or": [
        {
            "metadata_field": {
                <Operator>: <Value>
            }
        },
        {
            "metadata_field": {
                <Operator>: <Value>
            }
        }
    ]
}
```


In [22]:
results = await collection.query(
    query_texts=["中东非的智能手机出货量为多少部?"],
    n_results=20,
    where={
        "$and": [
            {
                "title": "科技行业2025年展望",
            },
            {
                "index": {
                    "$gt": 15
                }
            }
        ]
    }
)
results

{'ids': [['0fdfe513c3794565b865a104945105d5',
   '5d8b773d740c459aaa2ed01e8365d4b7',
   '8bafa6ead2aa4ec1b00a31b912d0ba86',
   'dc47b8a98d444ff8983f2eaaf6ce01d6',
   'ba044e3ef2ec4510b48d691cf81c75a2',
   '12b303af297347e299251f3d8a79a549',
   '982b696ee125406aabee32d3b86ffe62']],
 'distances': [[0.6475816981625376,
   0.6794282086911847,
   0.7959771981980448,
   0.8101529216182416,
   1.0816273847301765,
   1.138325951526666,
   1.1834027176394268]],
 'embeddings': None,
 'metadatas': [[{'author': '沈岱;马智焱;黄佳琦',
    'index': 22,
    'path': './科技行业 2025 年展望.txt',
    'time': '2024年12月13日',
    'title': '科技行业2025年展望'},
   {'author': '沈岱;马智焱;黄佳琦',
    'index': 16,
    'path': './科技行业 2025 年展望.txt',
    'time': '2024年12月13日',
    'title': '科技行业2025年展望'},
   {'author': '沈岱;马智焱;黄佳琦',
    'index': 21,
    'path': './科技行业 2025 年展望.txt',
    'time': '2024年12月13日',
    'title': '科技行业2025年展望'},
   {'author': '沈岱;马智焱;黄佳琦',
    'index': 20,
    'path': './科技行业 2025 年展望.txt',
    'time': '2024年12月

### 1.2.3 \\$in 和 \\$nin 查询

```python
# 1. $in
{
  "metadata_field": {
    "$in": ["value1", "value2", "value3"]
  }
}
# 2. $nin
{
  "metadata_field": {
    "$nin": ["value1", "value2", "value3"]
  }
}
```

In [23]:
results = await collection.query(
    query_texts=["中东非的智能手机出货量为多少部?"],
    n_results=20,
    where={
        "$and": [
            {
                "title": "科技行业2025年展望",
            },
            {
                "index": {
                    "$in": [15, 16, 17, 18, 19, 20]
                }
            }
        ]
    }
)
results

{'ids': [['5d8b773d740c459aaa2ed01e8365d4b7',
   'dc47b8a98d444ff8983f2eaaf6ce01d6',
   '0c86b133f7514665b4a8684211d2d3d1',
   'ba044e3ef2ec4510b48d691cf81c75a2',
   '12b303af297347e299251f3d8a79a549',
   '982b696ee125406aabee32d3b86ffe62']],
 'distances': [[0.6794282086911847,
   0.8101529216182416,
   0.9953305774166606,
   1.0816273847301765,
   1.138325951526666,
   1.1834027176394268]],
 'embeddings': None,
 'metadatas': [[{'author': '沈岱;马智焱;黄佳琦',
    'index': 16,
    'path': './科技行业 2025 年展望.txt',
    'time': '2024年12月13日',
    'title': '科技行业2025年展望'},
   {'author': '沈岱;马智焱;黄佳琦',
    'index': 20,
    'path': './科技行业 2025 年展望.txt',
    'time': '2024年12月13日',
    'title': '科技行业2025年展望'},
   {'author': '沈岱;马智焱;黄佳琦',
    'index': 15,
    'path': './科技行业 2025 年展望.txt',
    'time': '2024年12月13日',
    'title': '科技行业2025年展望'},
   {'author': '沈岱;马智焱;黄佳琦',
    'index': 19,
    'path': './科技行业 2025 年展望.txt',
    'time': '2024年12月13日',
    'title': '科技行业2025年展望'},
   {'author': '沈岱;马智焱;黄佳琦',

## 1.3 where_document

根据文档内容来过滤(关键字查询)

```json
{"$contains": "search_string"}
```


In [24]:
results = await collection.query(
    query_texts=["中东非的智能手机出货量为多少部?"],
    n_results=20,
    where={
        "title": "科技行业2025年展望",
    },
    where_document={
        "$contains": "中东非"
    }
)
results

{'ids': [['ba044e3ef2ec4510b48d691cf81c75a2',
   '982b696ee125406aabee32d3b86ffe62']],
 'distances': [[1.0816273847301765, 1.1834027176394268]],
 'embeddings': None,
 'metadatas': [[{'author': '沈岱;马智焱;黄佳琦',
    'index': 19,
    'path': './科技行业 2025 年展望.txt',
    'time': '2024年12月13日',
    'title': '科技行业2025年展望'},
   {'author': '沈岱;马智焱;黄佳琦',
    'index': 17,
    'path': './科技行业 2025 年展望.txt',
    'time': '2024年12月13日',
    'title': '科技行业2025年展望'}]],
 'documents': [['高端机型相对优异的表现与我们在安卓品牌端以及供应链端看到的情况类 似。虽然全球智能手机换机周期仍然处于偏高位置，但是在较长换机周期的 情况下，用户愿意提升预算购买相对高端的机型。而千元以下手机占比提升， 主要来自于中东非等发展地区手机出货量更加强劲的表现。这可能是美国进 入降息周期带动这些地区手机需求的释放。\n其中，小米的高端机型表现比较典型。今年三季度，小米 3,000 元人民币以 上的智能手机出货量占到小米总出货量的6.3%，同比增加 1.5 个百分点，环 比增加0.5 个百分点(图表 41)。',
   '从地区来看，中东非的出货量为 4,199 万部，同比增速为 14%，表现亮眼， 唯一一个达到双位数增速的地区。紧随其后的亚洲其他地区出货量4,502 万 部，同比增长8%。这两个区域的出货量表现好于我们此前预测。虽然印度 三季度的出货量同比增长5%，但是略微低于我们此前预测。\n在三季度，中国的出货量 6,878 万部，同比增长 3%，环比下滑 4%(图表32)。欧洲和北美发达市场的出货量分别同比下降 5%和 1%。这三个区域的 出货量表现大体符合我们此前的预测。']],
 'uris': None,
 'data': None,
 

# 2. chromadb-get()方法
全部都是可选参数
- ids: [],根据id获取文档片段信息
- where: 根据metadata过滤
- where_document: 根据关键字从文档字符串中过滤。
- limit: int, 限制多少条数据
- offset: int, 偏移量，从什么位置开始获取数据
- include: 默认: metadatas，documents, 可选值：documents/embeddings/metadatas/uris/data/(distances除外)

In [46]:
results = await collection.get(
   #  ids=['ba044e3ef2ec4510b48d691cf81c75a2',
   # '982b696ee125406aabee32d3b86ffe62'],
    offset=10,
    limit=10,
    include=["metadatas", "documents", "embeddings"]
)
results

{'ids': ['ccf537ac31cf4818a3b2434899a65e94',
  'b71b2441926a4697a2d3faf22688b479',
  'c348fd5ccd1447efbad1be45192649db',
  '34ad146d37a449c18b62dd8f528c71e4',
  'c549d0bc2d2f4505824171d42ac3681c',
  '0c86b133f7514665b4a8684211d2d3d1',
  '5d8b773d740c459aaa2ed01e8365d4b7',
  '982b696ee125406aabee32d3b86ffe62',
  '12b303af297347e299251f3d8a79a549',
  'ba044e3ef2ec4510b48d691cf81c75a2'],
 'embeddings': array([[-0.02395189,  0.06656551,  0.05398974, ...,  0.10957031,
          0.01413448,  0.01485522],
        [ 0.03590994,  0.03164856, -0.02578723, ...,  0.04841742,
         -0.00986954,  0.02870936],
        [-0.02215509,  0.06232987,  0.00581868, ...,  0.05894265,
          0.02718835,  0.05889684],
        ...,
        [ 0.05824693,  0.01147575,  0.10186285, ...,  0.01364945,
         -0.04274667,  0.02547619],
        [ 0.03472046, -0.01027943,  0.02211903, ..., -0.04201765,
         -0.03046407,  0.05962202],
        [ 0.03023288,  0.00027974,  0.05056516, ...,  0.00068486,
         

# 3. chromadb-添加/更新数据

- add(): 添加新数据
- update(): 根据id更新数据。
- upsert()：那么数据存在则更新，否则插入。

参数：
- ids: [],
- embeddings: [], embedding(嵌入)向量列表
- metadatas: [], 
- documents: [],
- images: [], 多模态图片数据，每个元素是一个表示图片的numpy列表
- uris: [], 和images配合使用


# 4. chromadb-delete() 删除数据

- ids: [],根据id删除文档信息
- where: 根据metadata删除
- where_document: 根据关键字从文档字符串中过滤

In [47]:
results = await collection.delete(
    ids=['ba044e3ef2ec4510b48d691cf81c75a2',
   '982b696ee125406aabee32d3b86ffe62']
)
results

In [48]:
results = await collection.get(
    ids=['ba044e3ef2ec4510b48d691cf81c75a2',
   '982b696ee125406aabee32d3b86ffe62']
)

In [49]:
results

{'ids': [],
 'embeddings': None,
 'metadatas': [],
 'documents': [],
 'data': None,
 'uris': None,
 'included': ['documents', 'metadatas']}

In [50]:
await collection.delete(
    where={
        "title": "DeepSeek突传大消息！最新回应来了"
    }
)


In [54]:
results = await collection.get()
results

{'ids': ['33bbdacd6c3f4a43bff29307eb8c9ad0',
  '82155abbe91d416b8e936d1596f3d159',
  '5167bd429e6748a19b3a911990475500',
  'ccb49d54bac24338a92373512e544be7',
  'a37fc789029740b28613b3959b4852d6',
  'a820f5a6e3244aa2a19a0fa9c64ed2bf',
  'abd7e0d2c7d2449f9a67c584673bbe6b',
  '00b9ee79053942bcaf4aaaeaa1a26fc4',
  '090a9bb3a06c44948b29814c29548246',
  '68d37f062d0a4213840d20eec87bfbe4',
  'ccf537ac31cf4818a3b2434899a65e94',
  'b71b2441926a4697a2d3faf22688b479',
  'c348fd5ccd1447efbad1be45192649db',
  '34ad146d37a449c18b62dd8f528c71e4',
  'c549d0bc2d2f4505824171d42ac3681c',
  '0c86b133f7514665b4a8684211d2d3d1',
  '5d8b773d740c459aaa2ed01e8365d4b7',
  '12b303af297347e299251f3d8a79a549',
  'dc47b8a98d444ff8983f2eaaf6ce01d6',
  '8bafa6ead2aa4ec1b00a31b912d0ba86',
  '0fdfe513c3794565b865a104945105d5'],
 'embeddings': None,
 'metadatas': [{'author': '沈岱;马智焱;黄佳琦',
   'index': 0,
   'path': './科技行业 2025 年展望.txt',
   'time': '2024年12月13日',
   'title': '科技行业2025年展望'},
  {'author': '沈岱;马智焱;黄佳琦',
   

In [55]:
await collection.delete(
    where_document={
        "$contains": "sia_huang@spdbi.com"
    }
)
# 33bbdacd6c3f4a43bff29307eb8c9ad0

In [56]:
await collection.get(ids=["33bbdacd6c3f4a43bff29307eb8c9ad0"])

{'ids': [],
 'embeddings': None,
 'metadatas': [],
 'documents': [],
 'data': None,
 'uris': None,
 'included': ['documents', 'metadatas']}

In [63]:
results = await collection.delete(
    ids=["33bbdacd6c3f4a43bff29307eb8c9ad0"]
)

In [64]:
results